# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 1:44PM,240415,10,0085798484,ISDIN Corporation,Released
1,Jun 24 2022 1:44PM,240415,10,0085798696,ISDIN Corporation,Released
2,Jun 24 2022 1:44PM,240415,10,0085799062,ISDIN Corporation,Released
3,Jun 24 2022 1:40PM,240417,10,MSP211599,"Methapharm, Inc.",Released
4,Jun 24 2022 1:37PM,240413,10,0085798919,ISDIN Corporation,Released
5,Jun 24 2022 1:37PM,240413,10,0085798918,ISDIN Corporation,Released
6,Jun 24 2022 1:37PM,240413,10,0085798920,ISDIN Corporation,Released
7,Jun 24 2022 1:37PM,240413,10,0085798915,ISDIN Corporation,Released
8,Jun 24 2022 1:37PM,240413,10,0085798958,ISDIN Corporation,Released
9,Jun 24 2022 1:37PM,240413,10,0085798953,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,240412,Executing,1
13,240412,Released,2
14,240413,Released,54
15,240415,Released,3
16,240417,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240409,1.0,NaN
240412,1.0,2.0
240413,NaN,54.0
240415,NaN,3.0
240417,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240323,1.0,0.0
240348,0.0,1.0
240352,0.0,1.0
240355,0.0,1.0
240384,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240323,1,0
240348,0,1
240352,0,1
240355,0,1
240384,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240323,1,0
1,240348,0,1
2,240352,0,1
3,240355,0,1
4,240384,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240323,1,
1,240348,,1
2,240352,,1
3,240355,,1
4,240384,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation
3,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc."
4,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation
58,Jun 24 2022 1:19PM,240412,12,Hush Hush
61,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc."
62,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation
97,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc."
99,Jun 24 2022 11:16AM,240397,10,"Methapharm, Inc."
106,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation
142,Jun 24 2022 9:18AM,240384,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation,,3
1,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc.",,1
2,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation,,54
3,Jun 24 2022 1:19PM,240412,12,Hush Hush,1,2
4,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc.",1,
5,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,,35
6,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc.",,2
7,Jun 24 2022 11:16AM,240397,10,"Methapharm, Inc.",,7
8,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,,36
9,Jun 24 2022 9:18AM,240384,10,Eywa Pharma Inc.,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation,3,
1,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc.",1,
2,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation,54,
3,Jun 24 2022 1:19PM,240412,12,Hush Hush,2,1
4,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc.",,1
5,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35,
6,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc.",2,
7,Jun 24 2022 11:16AM,240397,10,"Methapharm, Inc.",7,
8,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,36,
9,Jun 24 2022 9:18AM,240384,10,Eywa Pharma Inc.,1,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation,3,
1,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc.",1,
2,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation,54,
3,Jun 24 2022 1:19PM,240412,12,Hush Hush,2,1
4,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation,3.0,NaN
1,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc.",1.0,NaN
2,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation,54.0,NaN
3,Jun 24 2022 1:19PM,240412,12,Hush Hush,2.0,1.0
4,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 1:44PM,240415,10,ISDIN Corporation,3.0,0.0
1,Jun 24 2022 1:40PM,240417,10,"Methapharm, Inc.",1.0,0.0
2,Jun 24 2022 1:37PM,240413,10,ISDIN Corporation,54.0,0.0
3,Jun 24 2022 1:19PM,240412,12,Hush Hush,2.0,1.0
4,Jun 24 2022 11:50AM,240409,19,"GUSA Granules USA, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1682816,137.0,2.0
12,480797,3.0,1.0
16,961378,3.0,1.0
19,240409,0.0,1.0
20,240405,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1682816,137.0,2.0
1,12,480797,3.0,1.0
2,16,961378,3.0,1.0
3,19,240409,0.0,1.0
4,20,240405,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,137.0,2.0
1,12,3.0,1.0
2,16,3.0,1.0
3,19,0.0,1.0
4,20,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,3.0
2,16,Released,3.0
3,19,Released,0.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,2.0,1.0,1.0,1.0,0.0
Released,137.0,3.0,3.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,2.0,1.0,1.0,1.0,0.0
1,Released,137.0,3.0,3.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,2.0,1.0,1.0,1.0,0.0
1,Released,137.0,3.0,3.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()